# Poisson in 1D with scikit-fem

We solve the boundary value problem
$$-u''(x)=1,\quad x\in(0,1),\qquad u(0)=u(1)=0.$$

**Exact solution:** $u(x)=\tfrac{1}{2}x(1-x)$.
We use a P1 (linear) finite element discretization on a uniform 1D mesh.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from skfem import MeshLine, Basis, ElementLineP1, asm, condense
from skfem.models.poisson import laplace, unit_load

# Mesh and basis on [0, 1]; refine for more elements (2^refine elements).
refine = 6   # 64 elements; runs fast on Binder
m = MeshLine().refined(refine)
e = ElementLineP1()
basis = Basis(m, e)

# Assemble stiffness and load for -u'' = 1
A = asm(laplace, basis)
b = asm(unit_load, basis)

# Dirichlet boundary conditions at x=0 and x=1 (u=0).
try:
    D = basis.get_dofs().all()  # newer skfem API collects boundary DOFs
except Exception:
    # Fallback: detect endpoints by coordinate
    xnodes = m.p[0]
    tol = 1e-12
    D = np.where((np.abs(xnodes - 0.0) < tol) | (np.abs(xnodes - 1.0) < tol))[0]

# Apply Dirichlet BCs (u=0 at boundary) by condensation
Acond, bcond = condense(A, b, D=D)

# Solve linear system for free DOFs
u_free = np.linalg.solve(Acond, bcond)

# Reconstruct full solution vector with zeros at Dirichlet nodes
u = np.zeros(basis.N)
free = np.setdiff1d(np.arange(basis.N), np.atleast_1d(D))
u[free] = u_free

# Coordinates and exact solution
x = m.p[0]
u_exact = 0.5 * x * (1 - x)
err_inf = np.max(np.abs(u - u_exact))
print(f"Infinity-norm error: {err_inf:.3e}")

# Plot FE vs exact
plt.figure(figsize=(6,4))
plt.plot(x, u, marker='o', label='FE (P1)')
plt.plot(x, u_exact, linestyle='--', label='Exact 0.5·x·(1-x)')
plt.xlabel('x')
plt.ylabel('u(x)')
plt.title('1D Poisson: -u"=1 on (0,1), u(0)=u(1)=0')
plt.legend()
plt.tight_layout()
plt.show()

### Experiments
- Change `refine` and observe the error decrease.
- Replace the RHS with a function by defining a custom load form.
- Try nonzero Dirichlet data by modifying the condensation step.